In [18]:
import os

In [19]:
patent_abstract = """
Safety and stability control method against automobile tire blowout
Abstract
A safety and stability control method against automobile tire blowout, which is used for manned and unmanned driving vehicles and based on vehicle braking, driving, steering and suspension systems. The present method establishes tire blowout determination based on a tire pressure detection mode, a status tire pressure mode and a steering mechanics state mode, and uses a safety and stability control mode, model and algorithm, and control structure and process against automobile tire blowout. On the basis of a tire blowout state point, the vehicle braking, driving, steering, steering wheel steering force and suspension balancing control are carried out in a coordinated manner by entering and exiting a tire blowout control state and switching between a normal mode and a tire blowout control mode, so as to realize tire blowout control in which real or unreal tire blowout processes overlap. In cases where a tire blowout process state and the motion states of the wheel and vehicle with a blown tire are changed rapidly, the technical difficulties of the wheel and the vehicle being seriously unstable due to tire blowout and the extreme tire blowout state being difficult to control are overcome, solving the safety technical problems associated with automobile tire blowout.
"""


In [20]:
from openai import OpenAI
oai_client = OpenAI()

oai_client.embeddings.create(
        model="text-embedding-ada-002",
        input=patent_abstract
    )


CreateEmbeddingResponse(data=[Embedding(embedding=[-0.01479567401111126, 0.008381889201700687, 0.01342079322785139, -0.011410814709961414, -0.03129424527287483, 0.02693929336965084, -0.007453669793903828, 0.013532458804547787, -0.011215399950742722, -0.024315154179930687, -0.006162537727504969, 0.017308147624135017, 0.007823561318218708, 0.038329169154167175, -0.00024405011208727956, -0.013309127651154995, 0.03280172869563103, -0.0006948558147996664, 0.0025089834816753864, -0.0014699711464345455, -3.0206379960873164e-05, -0.0039257388561964035, -0.041483718901872635, -0.016805652529001236, -0.022374967113137245, -0.026087844744324684, 0.04483368247747421, -0.03514670580625534, 0.016205450519919395, -0.023714952170848846, 0.00994520541280508, -0.0026537994854152203, 0.01099206879734993, -0.023812660947442055, 0.007342004217207432, -0.035872530192136765, 0.010740822181105614, -0.035174619406461716, -0.001659278990700841, -0.0017866474809125066, 0.002791636623442173, 0.02721845731139183, 

In [21]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'),
                                             model_name="text-embedding-ada-002")


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Patents",
                                                      embedding_function=embedding_function)

In [22]:
vector_store.add("patent_info", documents=patent_abstract)

Add of existing embedding ID: patent_info
Insert of existing embedding ID: patent_info


In [23]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()
#tru.reset_database() #Reset the database. Clears all tables.


In [24]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
        query_texts=query,
        n_results=2
    )
        return results['documents'][0]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = oai_client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=
        [
            {"role": "user",
            "content": 
            f"The following abstract descripes a concept for a novel invention: \n"
            f"---------------------\n"
            f"{context_str}"
            f"\n---------------------\n"
            f"Given this information, please execute the following instructions: {query}"
            }
        ]
        ).choices[0].message.content
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

In [25]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np

# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


In [26]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG Key Words',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

Function <function RAG_from_scratch.generate_completion at 0x000001FA4C60C280> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001FA4E2F89D0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.query at 0x000001FA4C60C310> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001FA4E2F89D0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.retrieve at 0x000001FA4C60C1F0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001FA4E2F89D0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.


In [27]:
with tru_rag as recording:
    rag.query("Name 5 key words based on this abstract, that I can use for the search in a patent database. Optimize the key words to get back more results. Result as python string.")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [28]:
tru.get_leaderboard(app_ids=["RAG Key Words"])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
RAG Key Words,1.0,0.875,1.0,3.6,0.0


In [29]:

tru_rag = TruCustomApp(rag,
    app_id = 'RAG Classifications',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

Function <function RAG_from_scratch.generate_completion at 0x000001FA4C60C280> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001FA4E2F89D0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.query at 0x000001FA4C60C310> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001FA4E2F89D0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.retrieve at 0x000001FA4C60C1F0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001FA4E2F89D0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.


In [30]:
with tru_rag as recording:
    rag.query("Name 5 CPC classifications based on this abstract, that I can use for the search in a patent database. \
Please give me a python string for the codes of the 5 most relevant \
CPC classifications to a possible patent.")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [31]:
tru.get_leaderboard(app_ids=["RAG Classifications"])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
RAG Classifications,1.0,0.525,0.105,3.8,0.0


In [32]:

tru_rag = TruCustomApp(rag,
    app_id = 'RAG Patent Comparison',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

Function <function RAG_from_scratch.generate_completion at 0x000001FA4C60C280> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001FA4E2F89D0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.query at 0x000001FA4C60C310> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001FA4E2F89D0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.retrieve at 0x000001FA4C60C1F0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001FA4E2F89D0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.


In [33]:
with tru_rag as recording:
    rag.query(f"""
'The following texts are abstracts from patent specifications. Your task is to compare the novel invention from above to all the other abstracts from below. \n
It is important that you focus on comparing the concepts that the abstracts describe, not the way they are written. \n
Rank the remaining abstracts on how well they match with the Testing Abstract by giving them a rating from 0 to 10 points. \n        
0 meaning they have absolutely nothing in common and 10 meaning they basically describe the exact same idea.\n  
Your output should be a python dictionary with the title "comparison", each element hast the Abstract number as key and the rating as value.\n        
I want to convert your output string to an actual dictionary, so make sure the formatting is right.\n\n        
        
US20210213935A1: "A safety and stability control method against automobile tire blowout, which is used for manned and unmanned driving vehicles and based on vehicle braking, driving, steering and suspension systems. The present method establishes tire blowout determination based on a tire pressure detection mode, a status tire pressure mode and a steering mechanics state mode, and uses a safety and stability control mode, model and algorithm, and control structure and process against automobile tire blowout. On the basis of a tire blowout state point, the vehicle braking, driving, steering, steering wheel steering force and suspension balancing control are carried out in a coordinated manner by entering and exiting a tire blowout control state and switching between a normal mode and a tire blowout control mode, so as to realize tire blowout control in which real or unreal tire blowout processes overlap. In cases where a tire blowout process state and the motion states of the wheel and vehicle with a blown tire are changed rapidly, the technical difficulties of the wheel and the vehicle being seriously unstable due to tire blowout and the extreme tire blowout state being difficult to control are overcome, solving the safety technical problems associated with automobile tire blowout."\nCN110481541B: "The invention relates to a safe and stable control method for automobile tire burst, which is used for vehicles with people and unmanned vehicles, is based on vehicle braking, driving, steering and suspension systems, and belongs to the field of automobile tire burst safety. The method determines tire burst detection, tire pressure state and steering mechanical state mode, and adopts a safe and stable control mode, a model and an algorithm, a control structure and a flow of automobile tire burst; based on the tire burst state point, vehicle braking, driving, steering wheel rotating force and suspension balance control are coordinated through entering and exiting of tire burst control and conversion of normal and tire burst control modes, and real or non-real tire burst process overlapping tire burst control is achieved; under the condition that the tire burst process state, the tire burst wheel and the vehicle motion state change rapidly, important technical barriers that the tire burst of the wheel and the vehicle is seriously unstable, the tire burst extreme state is difficult to control and the like are broken through, and the important problem which puzzles the tire burst safety of the automobile for a long time at present is solved."\nCN110481540B: "The invention relates to a safe and stable control system for automobile tire burst, which is used for manned and unmanned vehicles, is based on vehicle braking, driving, steering and suspension systems, and belongs to the field of automobile tire burst safety. The system establishes tire burst judgment of detecting tire pressure, state tire pressure and steering mechanical state modes, and adopts an automobile tire burst safety and stability control mode, a model and an algorithm, a control structure and a flow; based on the tire burst state point, vehicle braking, driving, steering wheel rotating force and suspension balance control are coordinated through entering and exiting of tire burst control and conversion of normal and tire burst control modes, and real or non-real tire burst process overlapping tire burst control is achieved; under the condition that the tire burst process state, the tire burst wheel and the vehicle motion state change rapidly, important technical barriers that the tire burst of the wheel and the vehicle is seriously unstable, the tire burst extreme state is difficult to control and the like are broken through, and the important problem which puzzles the tire burst safety of the automobile for a long time at present is solved."\nUS20210188252A1: "Disclosed is a car flat tire safety and stability control method for manned and unmanned vehicles based on vehicle braking, driving, steering and suspension systems. The method establishes flat tire determination by tire pressure detection, a state tire pressure and a mechanical steering state, and adopts a car tire burst safety and stability control mode, model and algorithm, and a control structure and procedure. Based on a flat tire state point, the control over vehicle braking, driving and steering, a steering wheel gyroscopic force and suspension balancing is executed in a coordinated manner by means of switching between entering and exiting flat tire control and between a normal mode and a flat tire control mode, thereby realizing overlapped flat tire control of a real or unreal flat tire process. In the case of sharp changes in a flat tire process state, a flat tire wheel and a vehicle motion state, the technical problems of the severe instability of wheels and a vehicle due to a flat tire, the technical difficulties in controlling an extreme flat tire state are resolved, and the problem of the car flat tire safety technology is solved."\nUS11760339B2: "In some embodiments, a rapid-response active suspension system controls suspension force and position for improving vehicle safety and drivability. The system may interface with various sensors that detect safety critical vehicle states and adjust the suspension of each wheel to improve safety. Pre-crash and collision sensors may notify the active suspension controller of a collision and the stance may be adjusted to improve occupant safety during an impact while maintaining active control of the wheels. Wheel forces may also be controlled to improve the effectiveness of vehicle safety systems such as ABS and ESP in order to improve traction. Also, bi-directional information may be communicated between the active suspension system and other vehicle safety systems such that each system may respond to information provided to the other."\nCN105691381B: "The invention discloses a kind of four motorized wheels electric automobile stability control method, joint utilizes phasor and safe speed area judging stability, accurately judges whether current motoring condition is stablized in real time.The present invention proposes a kind of four motorized wheels electric automobile stabilitrak at the same time, it is the four motorized wheels system and four-wheel independent brake system of the method Comprehensive Control four motorized wheels electric automobile using optimal torque distribution, so that the work that four wheels are more coordinated, improves accuracy and practicality of the stability controller on four motorized wheels electric automobile."\nWO2020187259A1: "A safety monitoring method and system for an autonomous vehicle. Said method comprises: a lateral control safety monitoring step, establishing a correlation between a lateral acceleration of a vehicle, a longitudinal vehicle speed and a steering wheel steering angle, acquiring a maximum steering wheel steering angle corresponding to a given maximum lateral acceleration at different vehicle speeds, taking the maximum steering wheel steering angle as a steering angle threshold, determining whether the steering wheel steering angle is greater than the steering angle threshold, and if so, limiting the steering wheel steering angle within the steering angle threshold, otherwise, normally outputting; and a longitudinal control safety monitoring step, with regard to different longitudinal control states, acquiring an acceleration threshold corresponding to a given maximum target acceleration at different vehicle speeds, and determining whether the target acceleration is greater than the acceleration threshold, and if so, limiting the target acceleration within the acceleration threshold, otherwise, normally outputting. The safety monitoring method and system for an autonomous vehicle design a safety monitoring policy for an automated driving system from the perspective of lateral control or longitudinal control respectively, so that a more integrated design is achieved."\nCN202491633U: "The utility model relates to a safety wheel for a son-mother tire automobile. A son wheel rim structure for fixing a son tire is additionally arranged at the middle position of a standard wheel rim; two tires can be simultaneously arranged on one wheel rim, namely the son tire and an external mother tire. The son tire has a solid structure made of an elastic material, and the mother tire is positioned outside the son tire and is used for covering the son tire. Compressed gas is filled in a cavity between the mother tire and the son tire. The safety wheel structure for the son-mother tire automobile can ensure the comfort in running vehicles in the normal state, can reduce the damage caused by tire blowout under high-speed conditions, and ensures the safety of continuous driving of the vehicles when air in the tires is insufficient."\nUS20090266465A1: "The universal system and device of the present invention, once inserted in a conventional pneumatic tire, allows the driven vehicle to be operated in a normal operating condition, as well as in both a low tire pressure and in a run flat condition. For such purpose the tire embodies this device, spherically shaped and not limited in size or quantity. The device is manufactured from rubber, neoprene or equivalent, and in various shore hardness as needed, to permit installation in all tire shapes and sizes, and can be accommodated on all standard or special vehicle rim sizes or configurations. The device is adaptable to automobile, trucks, motorcycles, bicycles or any vehicle that utilizes pneumatic tires and their various uses, including freeway driving, off road, snow and ice, and will adapt to any new configurations of tires and rims such as the low profile tires."\nCN104080593A: "A self-sealing tyre for vehicle wheels is described, comprising: a carcass structure, a tread band applied in a radially external position to said carcass structure at least in a crown area of the tyre, a liner applied in a radially inner position to said carcass structure, a layer of sealing material applied in a radially inner position to said carcass structure and which extends axially at least in all the crown area of the tyre in which said sealing material comprises a partially chemically crosslinked composition comprising an unsaturated styrene thermoplastic elastomer."\n'
""")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [34]:
tru.get_leaderboard(app_ids=["RAG Patent Comparison"])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
RAG Patent Comparison,0.65,0.575,0.916667,10.2,0.0
